## This example show how to use use pytfa in order to calculate thermodynamic MFA and how to get optimal fluxes, concentrations and Gibbs energy

Imports and load data

In [1]:
from time import time

import pytfa
from cobra.io import load_json_model
from pytfa.io import load_thermoDB, read_lexicon, annotate_from_lexicon, read_compartment_data, apply_compartment_data

thermo_data = load_thermoDB('/data/thermo_data.thermodb')
cobra_model = load_json_model('data/models/iJO1366.json')

lexicon = read_lexicon('data/models/iJO1366/lexicon.csv')
compartment_data = read_compartment_data('data/models/iJO1366/compartment_data.json')

Using license file /Users/svevol/gurobi.lic
Academic license - for non-commercial use only


Initialize

In [1]:
import pandas as pd

In [2]:
mytfa = pytfa.ThermoModel(thermo_data, cobra_model)

annotate_from_lexicon(mytfa, lexicon)
apply_compartment_data(mytfa, compartment_data)

biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'

Read LP format model from file /var/folders/c1/00y46vps5tv8cw2xr_gsqf0dh403rh/T/tmpv6k9l6s7.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


2020-10-16 19:14:11,451 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Configuration

In [3]:
mytfa.name = 'tutorial_basics'
mytfa.objective = biomass_rxn
mytfa.solver.problem.Params.NumericFocus = 3
mytfa.solver.configuration.tolerances.feasibility = 1e-9
mytfa.solver.configuration.presolve = True

FBA Solution

In [4]:
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value

TFA conversion

In [5]:
## TFA conversion
mytfa.prepare()
mytfa.convert()

2020-10-16 19:14:11,753 - thermomodel_None - INFO - # Model preparation starting...
2020-10-16 19:14:14,657 - thermomodel_None - WARNING - Warning : NULL/U
2020-10-16 19:14:14,658 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,658 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,659 - thermomodel_None - WARNING - Warning : NULL/U
2020-10-16 19:14:14,660 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,664 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,918 - thermomodel_None - WARNING - Warning : NULL/U
2020-10-16 19:14:14,918 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,919 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,920 - thermomodel_None - WARNING - Warning : NULL/U
2020-10-16 19:14:14,923 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:14,924 - thermomodel_None - WARNING - Warning : NULL/L
2020-10-16 19:14:15,062 - thermomodel_None - WARNING

In [6]:
mytfa.print_info()

                           value
key                             
name             tutorial_basics
description      tutorial_basics
num constraints            15034
num variables              15538
num metabolites             1807
num reactions               2585
                           value
key                             
num metabolites(thermo)     1550
num reactions(thermo)       1824
pct metabolites(thermo)  85.7775
pct reactions(thermo)    70.5609


Optimality

In [7]:
tfa_solution = mytfa.optimize()
tfa_value = tfa_solution.objective_value

Relaxation

In [8]:
if tfa_value < 0.1:
    start = time()
    
    from pytfa.optim.relaxation import relax_dgo

    mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5*fba_value
    relaxed_model, slack_model, relax_table = relax_dgo(mytfa)

    original_model, mytfa = mytfa, relaxed_model
    
    elapsed_time = time()-start
    print(f'Elapsed time: {elapsed_time:.2f} s')

    print('Relaxation: ')
    print(relax_table)
    
    tfa_solution = mytfa.optimize()
    tfa_value = tfa_solution.objective_value

Read LP format model from file /var/folders/c1/00y46vps5tv8cw2xr_gsqf0dh403rh/T/tmpssk5fsat.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2020-10-16 19:15:09,022 - thermomodel_tutorial_basics - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Read LP format model from file /var/folders/c1/00y46vps5tv8cw2xr_gsqf0dh403rh/T/tmp31mmo4xp.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2020-10-16 19:15:26,531 - thermomodel_tutorial_basics - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2020-10-16 19:15:35,839 - thermomodel_tutorial_basics - INFO - Adding slack constraints
adding slacks: 100%|██████████| 1824/1824 [00:42<00:00, 42.91it/s]
2020-10-16 19:16:19,555 - thermomodel_tutorial_basics - INFO - Optimizing slack model
2020-10-16 19:16:21,883 - thermomodel_tutorial_basics - INFO - Extracting relaxation
applying slack: 100%|██████████| 2585/2585 [00:00<00:00, 66060.28it/s]
2020-10-16 19:16:22,025 - thermomodel_tutorial_basics - INFO - Testing relaxation


Elapsed time: 82.43 s
Relaxation: 
              lb_in      ub_in  lb_change  ub_change     lb_out     ub_out
AIRC3    -11.941085  -8.988685   0.000000   2.582409 -11.941085  -6.406276
ARGSL     16.770653  18.636729   2.184361   0.000000  14.586293  18.636729
ATPS4rpp  -9.518201  -8.083092   1.517942   0.000000 -11.036143  -8.083092
GALT1     67.754735  69.018800  54.942183   0.000000  12.812552  69.018800
HEPK2     44.872230  45.345690  32.059679   0.000000  12.812552  45.345690
HEPT1     25.810459  27.074524  12.997907   0.000000  12.812552  27.074524
HEPT2      7.030447   8.294512   0.695940   0.000000   6.334508   8.294512
MECDPS    26.135683  30.646876  11.549390   0.000000  14.586293  30.646876
MTHTHFSs  13.297862  14.814659   6.891586   0.000000   6.406276  14.814659


Report solution

In [9]:
print('FBA Solution found : {0:.5g}'.format(fba_value))
print('TFA Solution found : {0:.5g}'.format(tfa_value))

FBA Solution found : 0.814
TFA Solution found : 0.80312


Handy functions

In [10]:
from handy import *

In [11]:
get_fluxes(mytfa)

DM_4CRSOL                         0.000179
DM_5DRIB                          0.000186
DM_AACALD                         0.000000
DM_AMOB                           0.000002
DM_MTHTHF                         0.001076
DM_OXAM                           0.000000
Ec_biomass_iJO1366_WT_53p95M      0.803119
Ec_biomass_iJO1366_core_53p95M    0.000000
DM_12ppd-R_e                      0.000000
DM_12ppd-S_e                      0.000000
DM_14glucan_e                     0.000000
DM_15dap_e                        0.000000
DM_23camp_e                       0.000000
DM_23ccmp_e                       0.000000
DM_23cgmp_e                       0.000000
DM_23cump_e                       0.000000
DM_23dappa_e                      0.000000
DM_26dap-M_e                      0.000000
DM_2ddglcn_e                      0.000000
DM_34dhpac_e                      0.000000
DM_3amp_e                         0.000000
DM_3cmp_e                         0.000000
DM_3gmp_e                         0.000000
DM_3hcinnm_

In [12]:
get_log_concentrations(mytfa)

10fthf_c        -13.815511
12dgr120_c       -6.087440
12dgr140_c       -6.087440
12dgr141_c       -6.087440
12dgr160_c       -6.087440
12dgr161_c       -6.087440
12dgr180_c       -3.050017
12dgr181_c       -3.050017
12ppd-R_c       -13.815511
12ppd-S_c       -13.815511
13dpg_c          -4.836644
14dhncoa_c       -2.995732
14glucan_c      -13.815511
15dap_c          -5.206249
1ddecg3p_c      -13.815511
1hdec9eg3p_c    -13.815511
1hdecg3p_c      -13.815511
1odec11eg3p_c   -13.815511
1odecg3p_c      -13.815511
1pyr5c_c         -2.995732
1tdec7eg3p_c    -13.815511
1tdecg3p_c      -13.815511
23dappa_c        -6.058763
23ddhb_c         -2.995732
23dhacoa_c       -4.949195
23dhb_c          -2.995732
23dhba_c        -13.815511
23dhbzs_c        -2.995732
23dhdp_c         -2.995732
23dhmb_c        -13.815511
                   ...    
thymd_p         -13.815511
tma_p           -13.815511
tre_p            -2.995732
trp-L_p         -13.509176
tsul_p          -13.815511
ttdca_p          -2.995732
t

In [13]:
get_free_energies(mytfa)

12DGR120tipp     7.807372e+00
12DGR140tipp     9.355502e+00
12DGR141tipp     8.581437e+00
12DGR160tipp     1.090363e+01
12DGR161tipp     1.496075e+01
12DGR180tipp     1.366248e+01
12DGR181tipp     1.830731e+01
12PPDRtex        1.000197e-09
12PPDRtpp        3.548130e+00
12PPDStex        1.000197e-09
12PPDStpp        3.548130e+00
14GLUCANabcpp   -1.719413e+01
14GLUCANtexi     7.266723e-01
23CAMPtex       -8.180017e+00
23CCMPtex       -8.180017e+00
23CGMPtex       -8.180017e+00
23CUMPtex       -1.773741e+00
23DAPPAt2pp     -6.406276e+00
23DAPPAtex      -1.773741e+00
23PDE2pp        -1.051733e+01
23PDE4pp        -4.111054e+00
23PDE7pp        -4.111054e+00
23PDE9pp        -4.111054e+00
26DAHtex        -8.180017e+00
2AGPA120tipp     9.454273e+00
2AGPA140tipp     1.022834e+01
2AGPA141tipp    -6.406276e+00
2AGPA160tipp     9.203978e+00
2AGPA161tipp     8.822199e+00
2AGPA180tipp     1.118876e+01
                     ...     
URIH            -1.437096e+01
URIK2           -6.157866e+00
URIt2pp   